In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
%uv pip install datasets

Using Python 3.12.6 environment at: /usr/local
Resolved 31 packages in 2.36s
⠙ Preparing packages... (0/5)
⠙ Preparing packages... (0/5)
⠙ Preparing packages... (0/5)
⠙ Preparing packages... (0/5)
⠙ Preparing packages... (0/5)
⠙ Preparing packages... (0/5)
⠙ Preparing packages... (0/5)
⠙ Preparing packages... (0/5)
⠙ Preparing packages... (0/5)
⠙ Preparing packages... (0/5)
⠙ Preparing packages... (0/5)
multiprocess ------------------------------ 138.96 KiB/143.30 KiB
⠙ Preparing packages... (0/5)
multiprocess ------------------------------ 138.96 KiB/143.30 KiB
⠙ Preparing packages... (0/5)
multiprocess ------------------------------ 138.96 KiB/143.30 KiB
xxhash     ------------------------------ 14.91 KiB/189.82 KiB
⠙ Preparing packages... (0/5)
multiprocess ------------------------------ 138.96 KiB/143.30 KiB
xxhash     ------------------------------ 14.91 KiB/189.82 KiB
⠙ Preparing packages... (0/5)
dill       ------------------------------     0 B/116.86 KiB
multiprocess ---------

In [3]:
%uv pip install transformers

Using Python 3.12.6 environment at: /usr/local
Audited 1 package in 16ms
Note: you may need to restart the kernel to use updated packages.


In [4]:
%uv pip install evaluate

Using Python 3.12.6 environment at: /usr/local
Resolved 32 packages in 164ms
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
Prepared 1 package in 36ms
Installed 1 package in 95ms
 + evaluate==0.4.6
Note: you may need to restart the kernel to use updated packages.


## Data Preparation - Preprocessing

In [16]:
import random
import hashlib

from datasets import load_dataset

BRIEF_PROMPTS = {
    'default': "Answer the following question as briefly as possible.\n",
    'chat': 'Answer the following question in a single brief but complete sentence.\n'}

md5hash = lambda s: str(int(hashlib.md5(s.encode('utf-8')).hexdigest(), 16))


def construct_few_shot_prompt_from_indices(dataset, example_indices, brief, brief_always, make_prompt):
    """
        Given a dataset and indices, construct a few-shot prompt.
        :param dataset:
        :param example_indices:
        :param brief:
        :param brief_always:
        :param make_prompt:

    """
    if not brief_always:
        prompt = brief
    else:
        prompt = ''

    for example_index in example_indices:
        example = dataset[example_index]
        if 'context' in example.keys():
            context = example["context"]
        else:
            context = None

        question = example["question"]
        answer = example["numerical_answer"]

        prompt = prompt + make_prompt(context, question, answer, brief, brief_always)

    return prompt


def get_make_prompt(prompt_type, use_context):
    if prompt_type == 'default':
        def make_prompt(context, question, answer, brief, brief_always):
            prompt = ''
            if brief_always:
                prompt += brief
            if use_context and (context is not None):
                prompt += f"Context: {context}\n"
            prompt += f"Question: {question}\n"
            if answer:
                prompt += f"Answer: {answer}\n\n"
            else:
                prompt += 'Answer:'
            return prompt
    else:
        raise ValueError

    return make_prompt


def split_dataset(dataset):
    """
        Get indices of answerable and unanswerable questions.
        :param dataset:

    """

    answerable_indices = [i for i, example in enumerate(dataset) if example['answer_length'] > 0]
    unanswerable_indices = [i for i, example in enumerate(dataset) if example['answer_length'] == 0]

    return answerable_indices, unanswerable_indices


def dataset_pre_process(example):
    example['answer_explanation'] = example['answer']

    answer_str = example['answer'].split('#### ')[-1].strip().replace(',', '')
    int_answer = int(answer_str)
    fl_answer = float(answer_str)

    example['id'] = md5hash(str(example['question']))
    example['answer_length'] = len(answer_str)

    if int_answer == fl_answer:
        example['answer_numerical_type'] = 'int'
        example['numerical_answer'] = int_answer
    else:
        example['answer_numerical_type'] = 'float'
        example['numerical_answer'] = fl_answer

    return example


def load_ds(dataset_name):
    """
        Load dataset

    """
    train_dt, validation_dt = None, None
    if dataset_name == "gsm8k":
        dataset = load_dataset("openai/gsm8k", "main")
        dataset = dataset.map(dataset_pre_process)

        dataset = dataset.filter(lambda x: x['id']!='229204758988173734073362288035613840709')

        test_valid_split = dataset['test'].train_test_split(test_size=0.3)
        train_dt = dataset["train"]
        validation_dt = test_valid_split["test"]

    return train_dt.select(range(2000)), validation_dt


def get_dataset(dataset_name, experiment_details, num_few_shot, running_parameters):
    """

    :param dataset_name:
    :param experiment_details:
    :param num_few_shot:
    :param running_parameters: brief_prompt, brief_always, enable_brief, use_context
    :return:
    """

    train_dataset, validation_dataset = load_ds(dataset_name)
    answerable_indices, unanswerable_indices = split_dataset(train_dataset)

    prompt_indices = random.sample(answerable_indices, num_few_shot)
    experiment_details['prompt_indices'] = prompt_indices
    remaining_answerable = list(set(answerable_indices) - set(prompt_indices))

    BRIEF = BRIEF_PROMPTS[running_parameters['brief_prompt']]
    arg = running_parameters['brief_always'] if running_parameters['enable_brief'] else True
    make_prompt = get_make_prompt(prompt_type='default', use_context=running_parameters['use_context'])
    prompt = construct_few_shot_prompt_from_indices(train_dataset, prompt_indices, BRIEF, arg, make_prompt)

    experiment_details['BRIEF'] = BRIEF
    experiment_details['prompt'] = prompt

    return train_dataset, validation_dataset, answerable_indices, unanswerable_indices, remaining_answerable, experiment_details

## Testing Code
# experiment_details = {}
# running_parameters = {'brief_prompt': 'default', 'brief_always': False, 'enable_brief': True, 'use_context': False}
# tr_dt, vl_dt, ans_ind, unans_ind, rm_answ, exp = get_dataset('gsm8k', experiment_details, 5, running_parameters)

## Utilities

In [6]:
import pickle
from evaluate import load


def get_reference(example):
    if 'numerical_answer' not in example:
        example = example['answer_explanation']

    answers = example['numerical_answer']

    answer_starts = example['answer'].rfind(str(answers))

    if answer_starts == -1:
        answer_starts = []

    reference = {'answers': {'answer_start': [answer_starts], 'text': [str(example['numerical_answer'])]}, 'id': example['id']}
    return reference


def get_metric(metric):
    if metric == 'squad':

        squad_metric = load("squad_v2")

        def metric(response, example, *args, **kwargs):
            # Compatibility with recomputation.
            if 'id' in example:
                exid = example['id']
            elif 'id' in example['reference']:
                exid = example['reference']['id']
            else:
                raise ValueError

            prediction = {'prediction_text': response, 'no_answer_probability': 0.0, 'id': exid}
            results = squad_metric.compute(
                predictions=[prediction],
                references=[get_reference(example)])
            return 1.0 if (results['f1'] >= 50.0) else 0.0

    return metric


def save(object, file):
    with open(f'{file}', 'wb') as f:
        pickle.dump(object, f)

## Model Initialisation and Utilities

In [7]:
import torch
from transformers import StoppingCriteria, StoppingCriteriaList
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer

stop_sequences = ['\n\n\n\n', '\n\n\n', 'Question:', 'Context:']
# '\n\n', '\n',


class StoppingCriteriaSub(StoppingCriteria):
    """
        Stop generations when they match a particular text or token.
    """

    def __init__(self, stops, tokenizer, match_on='text', initial_length=None):
        super().__init__()
        self.stops = stops
        self.initial_length = initial_length
        self.tokenizer = tokenizer
        self.match_on = match_on
        if self.match_on == 'tokens':
            self.stops = [torch.tensor(self.tokenizer.encode(i))  # .to('cuda')
                          for i in self.stops]
            print(self.stops)

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs):
        del scores
        for stop in self.stops:
            if self.match_on == 'text':
                generation = self.tokenizer.decode(input_ids[0][self.initial_length:], skip_special_tokens=False)
                match = stop in generation
            elif self.match_on == 'tokens':
                # Can be dangerous due to tokenizer ambiguities.
                match = stop in input_ids[0][-len(stop):]
            else:
                raise
            if match:
                return True
        return False


def model_init(model_name):
    """
        Model Initialization

        :param model_name:
        :return:
    """
    tokenizer, model = None, None
    if model_name == 'Mistral-7B-v0.1':
        torch.cuda.empty_cache()

        tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1", device_map="auto",
                                                  token_type_ids=None, clean_up_tokenization_spaces=False)
        model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1", device_map="auto",
                                                     attn_implementation='eager', max_memory={0: '80GIB'})

    return tokenizer, model


def model_predict(model, tokenizer, input_data, temperature, max_new_tokens, return_latent=False):

    token_limit = 2048
    stop_seq = stop_sequences + [tokenizer.eos_token]

    inputs = tokenizer(input_data, return_tensors="pt").to("cuda:0")
    

    if 'token_type_ids' in inputs:  # HF models seems has changed.
        del inputs['token_type_ids']
        pad_token_id = tokenizer.eos_token_id
    else:
        pad_token_id = None

    pad_token_id = tokenizer.eos_token_id

    if stop_seq is not None:
        stopping_criteria = StoppingCriteriaList([StoppingCriteriaSub(
            stops=stop_seq,
            initial_length=len(inputs['input_ids'][0]),
            tokenizer=tokenizer)])
    else:
        stopping_criteria = None

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            return_dict_in_generate=True,
            output_scores=True,
            output_hidden_states=True,
            temperature=temperature,
            do_sample=True,
            stopping_criteria=stopping_criteria,
            pad_token_id=pad_token_id,
        )

    if len(outputs.sequences[0]) > token_limit:
        raise ValueError(
            'Generation exceeding token limit %d > %d', len(outputs.sequences[0]), token_limit)

    full_answer = tokenizer.decode(
        outputs.sequences[0], skip_special_tokens=True)

    if full_answer.startswith(input_data):
        input_data_offset = len(input_data)
    else:
        input_data_offset = 0

    # Remove input from answer.
    answer = full_answer[input_data_offset:]

    # Remove stop_words from answer.
    stop_at = len(answer)
    sliced_answer = answer

    if stop_seq is not None:
        for stop in stop_seq:
            if answer.endswith(stop):
                stop_at = len(answer) - len(stop)
                sliced_answer = answer[:stop_at]
                break
        if not all([stop not in sliced_answer for stop in stop_seq]):
            error_msg = 'Error: Stop words not removed successfully!'
            error_msg += f'Answer: >{answer}< '
            error_msg += f'Sliced Answer: >{sliced_answer}<'
            print(error_msg)

    # Remove whitespaces from answer (in particular from beginning.)
    sliced_answer = sliced_answer.strip()
    token_stop_index = tokenizer(full_answer[:input_data_offset + stop_at], return_tensors="pt")['input_ids'].shape[1]
    n_input_token = len(inputs['input_ids'][0])
    n_generated = token_stop_index - n_input_token

    return_latent = True

    if n_generated == 0:
        print('Only stop_words were generated. For likelihoods and embeddings, taking stop word instead.')
        n_generated = 1

    if 'decoder_hidden_states' in outputs.keys():
        hidden = outputs.decoder_hidden_states
    else:
        hidden = outputs.hidden_states

    if len(hidden) == 1:
        last_input = hidden[0]
    elif (n_generated - 1) >= len(hidden):
        # if access idx is larger/equal
        last_input = hidden[-1]
    else:
        last_input = hidden[n_generated - 1]

    last_layer = last_input[-1]
    last_token_embedding = last_layer[:, -1, :].cpu()

    if return_latent:
        # Stack second last token embeddings from all layers
        if len(hidden) == 1:
            sec_last_input = hidden[0]
        elif (n_generated - 2) >= len(hidden):
            sec_last_input = hidden[-2]
        else:
            sec_last_input = hidden[n_generated - 2]
        sec_last_token_embedding = torch.stack([layer[:, -1, :] for layer in sec_last_input]).cpu()

        # Get the last input token embeddings (before generated tokens)
        last_tok_bef_gen_input = hidden[0]
        last_tok_bef_gen_embedding = torch.stack([layer[:, -1, :] for layer in last_tok_bef_gen_input]).cpu()

    # Get log_likelihoods.
    transition_scores = model.compute_transition_scores(outputs.sequences, outputs.scores, normalize_logits=True)
    log_likelihoods = [score.item() for score in transition_scores[0]]

    if len(log_likelihoods) == 1:
        log_likelihoods = log_likelihoods
    else:
        log_likelihoods = log_likelihoods[:n_generated]

    if len(log_likelihoods) == 0:
        raise ValueError

    hidden_states = (last_token_embedding,)

    if return_latent:
        hidden_states += (sec_last_token_embedding, last_tok_bef_gen_embedding)
    else:
        hidden_states += (None, None)

    return_values = (sliced_answer, log_likelihoods, hidden_states)

    return return_values


def get_p_true(model, tokenizer, input_data):
    """
        Get the probability of the model answering A (True) for the given input
        
        :param input_data:
    """

    input_data += ' A'
    tokenized_prompt_true = tokenizer(input_data, return_tensors='pt').to('cuda')['input_ids']

    target_ids_true = tokenized_prompt_true.clone()
    # Set all target_ids except the last one to -100.
    target_ids_true[0, :-1] = -100

    with torch.no_grad():
        model_output_true = model(tokenized_prompt_true, labels=target_ids_true)

    loss_true = model_output_true.loss

    return -loss_true.item()
    

def calculate_perplexity(input_data, model, tokenizer):
    """

        :param input_data:
        :param model:
        :param tokenizer:
        :return:
    """

    tokenized_data = tokenizer(input_data, return_tensors='pt').to('cuda')['input_ids']

    with torch.no_grad():
        model_output_true = model(tokenized_data, labels=tokenized_data)

    perplexity = - model_output_true.loss.item()
    return perplexity

## P True Data

In [8]:
from evaluate import load

squad_metric = load("squad_v2")


def p_true_construct_few_shot_prompt(
        *, model, tokenizer, dataset, indices, prompt, brief, brief_always, make_prompt,
        num_generations, metric, max_new_tokens, token_limit=2048):
    """
        Construct few shot prompt for p_true uncertainty metric.
    """

    # Call model n_shots many times
    few_shot_prompt = []
    all_responses = dict()

    for it, i in enumerate(indices):
        prompt_candidate = []
        example = dataset[i]
        question = example["question"]
        context = None

        if it != 0:
            prompt_candidate += ['\n']
        prompt_candidate += ['Question: ' + question]
        prompt_candidate += ['\nBrainstormed Answers: ']
        current_question = make_prompt(context, question, None, brief, brief_always)
        local_prompt = prompt + current_question

        responses = []
        for j in range(num_generations + 1):

            if j == 0:
                temperature = 0.1
            else:
                temperature = 1.0

            #response, _, _ = model.predict(local_prompt, temperature)
            response, _, _ = model_predict(model, tokenizer, local_prompt, temperature, max_new_tokens)

            responses.append(response)
            prompt_candidate += [f'{response.strip()} \n']
            if j == 0:
                # Save most likely response and compute correctness metric for it.
                most_likely_response = response
                is_correct = metric(response, example, model)
                answers = [str(example['numerical_answer'])]
                #[answer for answer in example['answers']['text']]

        all_responses[i] = dict(
            responses=responses, most_likely_response=most_likely_response,
            is_correct=is_correct)

        prompt_candidate += ['Possible answer: ' + most_likely_response + '\n']
        prompt_candidate += ['Is the possible answer:\n']
        prompt_candidate += ['A) True\n']
        prompt_candidate += ['B) False\n']
        prompt_candidate += ['The possible answer is:']
        prompt_candidate += [' A' if is_correct else ' B']

        prompt_len = len(tokenizer.encode(''.join(few_shot_prompt + prompt_candidate)))
        # At test time, get a maximum of `num_generations * model.token_limit` extra tokens
        # 200 buffer for question and 'Possible Answer'.
        max_input_len = prompt_len + num_generations * max_new_tokens + 200

        if max_input_len < token_limit:
            few_shot_prompt.extend(prompt_candidate)
        else:
            print('Cutting of p_true prompt at length %d.', it)
            break

    return ''.join(few_shot_prompt), all_responses, it


def calculate_p_true(
        model, tokenizer, question, most_probable_answer, brainstormed_answers,
        few_shot_prompt, hint=False):
    """
        Calculate p_true uncertainty metric.
    """

    if few_shot_prompt:
        prompt = few_shot_prompt + '\n'
    else:
        prompt = ''

    prompt += 'Question: ' + question
    prompt += '\nBrainstormed Answers: '
    for answer in brainstormed_answers + [most_probable_answer]:
        prompt += answer.strip() + '\n'
    prompt += 'Possible answer: ' + most_probable_answer + '\n'
    if not hint:
        prompt += 'Is the possible answer:\n'
        prompt += 'A) True\n'
        prompt += 'B) False\n'
        prompt += 'The possible answer is:'
    else:
        prompt += 'Do the brainstormed answers match the possible answer? Respond with A if they do, if they do not respond with B. Answer:'

    log_prob = get_p_true(model, tokenizer, prompt)

    return log_prob

In [18]:
import gc
import os
import torch
import random
import logging
import numpy as np
from tqdm import tqdm
#from model_utils import model_init
#from utils import get_metric, get_reference, save
#from dataset_prep import get_dataset, get_make_prompt, split_dataset


def main(args, tokenizer, model):
    experiment_details = {'args': args}
    random.seed(10)

    # todo - clean this up
    metric = get_metric('squad')

    dataset_name = 'gsm8k'
    answerable_only = False
    compute_p_true = True
    brief_always = False
    enable_brief = True
    num_few_shot = 5
    p_true_num_fewshot = 20
    num_generations = 10
    get_training_set_generations = True
    num_samples = 100
    get_training_set_generations_most_likely_only = True
    temperature = 1.0
    max_new_tokens = 50
    compute_accuracy_at_all_temps = True
    p_true_hint = False

    running_parameters = {'brief_prompt': 'default', 'brief_always': False,
                          'enable_brief': True, 'use_context': False}

    train_dataset, validation_dataset, answerable_indices, unanswerable_indices, remaining_answerable, experiment_details = get_dataset(
        dataset_name, experiment_details, num_few_shot, running_parameters)

    if answerable_only:
        unanswerable_indices = []
        val_answerable, val_unanswerable = split_dataset(validation_dataset)
        del val_unanswerable
        validation_dataset = [validation_dataset[i] for i in val_answerable]

    # Initialize model.
    model_name = 'Mistral-7B-v0.1'
    #tokenizer, model = model_init(model_name)
    make_prompt = get_make_prompt(prompt_type='default', use_context=running_parameters['use_context'])

    # Initialize prompt for p_true baseline.
    if compute_p_true:
        p_true_indices = random.sample(answerable_indices, p_true_num_fewshot)

        print(p_true_indices)

        remaining_answerable = list(set(remaining_answerable) - set(p_true_indices))
        make_prompt = get_make_prompt(prompt_type='default', use_context=running_parameters['use_context'])

        # todo
        p_true_few_shot_prompt, p_true_responses, len_p_true = p_true_construct_few_shot_prompt(
            model=model, tokenizer=tokenizer, dataset=train_dataset, indices=p_true_indices,
            prompt=experiment_details['prompt'], brief=experiment_details['BRIEF'],
            brief_always=brief_always and enable_brief,
            make_prompt=make_prompt, num_generations=num_generations,
            metric=metric, max_new_tokens=max_new_tokens)

        print(p_true_few_shot_prompt, p_true_responses, len_p_true)

        experiment_details['p_true_indices'] = p_true_indices
        experiment_details['p_true_responses'] = p_true_responses
        experiment_details['p_true_few_shot_prompt'] = p_true_few_shot_prompt

    # Start answer generation.
    for dataset_split in ['train', 'validation']:

        # This will store all input data and model predictions.
        accuracies, generations, results_dict, p_trues = [], {}, {}, []

        if dataset_split == 'train':
            if not get_training_set_generations:
                continue
            dataset = train_dataset
            possible_indices = list(set(remaining_answerable) | set(unanswerable_indices))

        else:
            dataset = validation_dataset
            possible_indices = range(0, len(dataset))

        # Evaluate over random subset of the datasets.
        try:
            indices = random.sample(possible_indices, min(num_samples, len(dataset)))
        except:
            indices = random.sample(possible_indices, min(num_samples, len(dataset), len(possible_indices)))
            
        experiment_details[dataset_split] = {'indices': indices}

        if num_samples > len(dataset):
            print('Not enough samples in dataset. Using all %d samples.', len(dataset))

        it = 0
        for index in tqdm(indices):
            if (it + 1 % 10) == 0:
                gc.collect()
                torch.cuda.empty_cache()
            it += 1

            # Grab example at index.
            example = dataset[index]
            question, context = example["question"], None
            generations[example['id']] = {'question': question, 'context': context}
            correct_answer = example['numerical_answer']

            current_input = make_prompt(
                context, question, None, experiment_details['BRIEF'], brief_always and enable_brief)
            local_prompt = experiment_details['prompt'] + current_input

            full_responses = []

            # We sample 1 low temperature answer on which we will compute the
            # accuracy and num_generation high temperature answers which will
            # be used to estimate the entropy.

            if dataset_split == 'train' and get_training_set_generations_most_likely_only:
                num_generations = 1
            else:
                num_generations = num_generations + 1

            for i in range(num_generations):

                # Temperature for first generation is always `0.1`.
                temperature = 0.1 if i == 0 else temperature

                predicted_answer, token_log_likelihoods, (
                embedding, emb_last_before_gen, emb_before_eos) = model_predict(model, tokenizer, local_prompt, 
                                                                                temperature, max_new_tokens, 
                                                                                return_latent=True)

                # Last token embedding
                embedding = embedding.cpu() if embedding is not None else None
                emb_last_before_gen = emb_last_before_gen.cpu() if emb_last_before_gen is not None else None
                emb_before_eos = emb_before_eos.cpu() if emb_before_eos is not None else None

                compute_acc = compute_accuracy_at_all_temps or (i == 0)
                if correct_answer and compute_acc:
                    acc = metric(predicted_answer, example, model)
                else:
                    acc = 0.0  # pylint: disable=invalid-name

                if i == 0:
                    accuracies.append(acc)
                    most_likely_answer_dict = {
                        'response': predicted_answer,
                        'token_log_likelihoods': token_log_likelihoods,
                        'embedding': embedding,
                        'accuracy': acc,
                        'emb_last_tok_before_gen': emb_last_before_gen,
                        'emb_tok_before_eos': emb_before_eos,
                    }

                    generations[example['id']].update({
                        'most_likely_answer': most_likely_answer_dict,
                        'reference': get_reference(example),
                    })
                else:
                    # Aggregate predictions over num_generations.
                    full_responses.append(
                        (predicted_answer, token_log_likelihoods, embedding, acc))

            # Append all predictions for this example to `generations`.
            generations[example['id']]['responses'] = full_responses

            if compute_p_true and dataset_split == 'validation':
                # Already compute p_true here. Avoid cost of generations in compute_uncertainty script.
                p_true = calculate_p_true(
                    model, tokenizer, question, most_likely_answer_dict['response'],
                    [r[0] for r in full_responses], p_true_few_shot_prompt,
                    hint=p_true_hint)
                p_trues.append(p_true)

        # Save generations for that split.
        save(generations, f'{dataset_split}_generations.pkl')

        # Log overall accuracy.
        accuracy = np.mean(accuracies)
        print(f"Overall {dataset_split} split accuracy: {accuracy}")

        if dataset_split == 'validation':
            if compute_p_true:
                results_dict['uncertainty_measures'] = {
                    'p_false': [1 - p for p in p_trues],
                    'p_false_fixed': [1 - np.exp(p) for p in p_trues],
                }

            print(results_dict)
            save(results_dict, 'uncertainty_measures.pkl')

    save(experiment_details, 'experiment_details.pkl')
    del model
    del tokenizer

In [10]:
model_name = 'Mistral-7B-v0.1'
tokenizer, model = model_init(model_name)

tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

## P ik Code

In [11]:
%uv pip install scikit-learn

Using Python 3.12.6 environment at: /usr/local
Audited 1 package in 19ms
Note: you may need to restart the kernel to use updated packages.


In [12]:
import os

import torch
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


def get_p_ik(train_embeddings, is_false, eval_embeddings=None, eval_is_false=None):
    """
        Fit linear classifier to embeddings to predict model correctness.
    """

    print('Accuracy of model on Task: %f.', 1 - torch.tensor(is_false).mean())

    # Convert the list of tensors to a 2D tensor.
    train_embeddings_tensor = torch.cat(train_embeddings, dim=0)
    embeddings_array = train_embeddings_tensor.cpu().numpy()

    # Split the data into training and test sets.
    X_train, X_test, y_train, y_test = train_test_split(embeddings_array, is_false,
                                                        test_size=0.2, random_state=42)

    # Fit a logistic regression model.
    model = LogisticRegression()
    model.fit(X_train, y_train)

    # Predict deterministically and probabilistically and compute accuracy and auroc for all splits.
    X_eval = torch.cat(eval_embeddings, dim=0).cpu().numpy()
    y_eval = eval_is_false

    Xs = [X_train, X_test, X_eval]
    ys = [y_train, y_test, y_eval]
    suffixes = ['train_train', 'train_test', 'eval']

    metrics, y_preds_proba = {}, {}
    fig, axes = plt.subplots(1, 3, figsize=(20, 8))
    fig.suptitle('Log Probabilities Predicted by p_ik when the true label is [False/True]')

    for ax, suffix, X, y_true in zip(axes, suffixes, Xs, ys):

        if suffix == 'eval':
            model = LogisticRegression()
            model.fit(embeddings_array, is_false)
            convergence = {'n_iter': model.n_iter_[0], 'converged': (model.n_iter_ < model.max_iter)[0]}

        y_pred = model.predict(X)
        y_pred_proba = model.predict_proba(X)
        y_preds_proba[suffix] = y_pred_proba
        acc_p_ik_train = accuracy_score(y_true, y_pred)
        auroc_p_ik_train = roc_auc_score(y_true, y_pred_proba[:, 1])
        split_metrics = {
            f'acc_p_ik_{suffix}': acc_p_ik_train,
            f'auroc_p_ik_{suffix}': auroc_p_ik_train}
        metrics.update(split_metrics)

        # Plotting.
        probabilities_of_false_points = y_pred_proba[:, 1][np.array(y_true) == 1.0]
        probabilities_of_true_points = y_pred_proba[:, 1][np.array(y_true) == 0.0]
        ax.hist(probabilities_of_false_points, bins=20, alpha=0.5, label='False')
        ax.hist(probabilities_of_true_points, bins=20, alpha=0.5, label='True')
        ax.legend(loc='upper right', title='True Label')
        fmt = {k: f"{v:.2f}" for k, v in split_metrics.items()}
        ax.set_title(f'Set: {suffix} \n {fmt}')

    # Plotting.
    axes[0].set_ylabel('Counts')
    axes[1].set_xlabel('Predicted Probabilities')
    os.system('mkdir -p figures')
    plt.savefig('figures/p_ik.png')
    plt.savefig('figures/p_ik.pdf')

    print('Metrics for p_ik classifier: %s.', metrics)

    return y_preds_proba['eval'][:, 1]

## Semantic Entropy

In [13]:
import os
import torch
import pickle
import random
import numpy as np
import torch.nn.functional as F

from transformers import AutoModelForSequenceClassification, AutoTokenizer

random.seed(10)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


class BaseEntailment:

    def save_prediction_cache(self):
        pass


class EntailmentDeberta(BaseEntailment):
    def __init__(self):
        self.tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v2-xlarge-mnli")
        self.model = AutoModelForSequenceClassification.from_pretrained(
            "microsoft/deberta-v2-xlarge-mnli").to(DEVICE)

    def check_implication(self, text1, text2, *args, **kwargs):
        # Deberta-mnli returns `neutral` and `entailment` classes at indices 1 and 2.
        inputs = self.tokenizer(text1, text2, return_tensors="pt").to(DEVICE)
        outputs = self.model(**inputs)
        logits = outputs.logits

        largest_index = torch.argmax(F.softmax(logits, dim=1))  # pylint: disable=no-member
        prediction = largest_index.cpu().item()

        if os.environ.get('DEBERTA_FULL_LOG', False):
            print('Deberta Input: %s -> %s', text1, text2)
            print('Deberta Prediction: %s', prediction)

        return prediction


def context_entails_response(context, responses, model):
    votes = []
    for response in responses:
        votes.append(model.check_implication(context, response))
    return 2 - np.mean(votes)


def get_semantic_ids(strings_list, model, strict_entailment=False, example=None):
    """Group list of predictions into semantic meaning."""

    def are_equivalent(text1, text2):

        implication_1 = model.check_implication(text1, text2, example=example)
        implication_2 = model.check_implication(text2, text1, example=example)  # pylint: disable=arguments-out-of-order
        assert (implication_1 in [0, 1, 2]) and (implication_2 in [0, 1, 2])

        if strict_entailment:
            semantically_equivalent = (implication_1 == 2) and (implication_2 == 2)

        else:
            implications = [implication_1, implication_2]
            # Check if none of the implications are 0 (contradiction) and not both of them are neutral.
            semantically_equivalent = (0 not in implications) and ([1, 1] != implications)

        return semantically_equivalent

    # Initialise all ids with -1.
    semantic_set_ids = [-1] * len(strings_list)
    # Keep track of current id.
    next_id = 0
    for i, string1 in enumerate(strings_list):
        # Check if string1 already has an id assigned.
        if semantic_set_ids[i] == -1:
            # If string1 has not been assigned an id, assign it next_id.
            semantic_set_ids[i] = next_id
            for j in range(i + 1, len(strings_list)):
                # Search through all remaining strings. If they are equivalent to string1, assign them the same id.
                if are_equivalent(string1, strings_list[j]):
                    semantic_set_ids[j] = next_id
            next_id += 1

    assert -1 not in semantic_set_ids

    return semantic_set_ids


def logsumexp_by_id(semantic_ids, log_likelihoods, agg='sum'):
    """
        Sum probabilities with the same semantic id.

        Log-Sum-Exp because input and output probabilities in log space.
    """
    unique_ids = sorted(list(set(semantic_ids)))
    assert unique_ids == list(range(len(unique_ids)))
    log_likelihood_per_semantic_id = []

    for uid in unique_ids:
        id_indices = [pos for pos, x in enumerate(semantic_ids) if x == uid]
        id_log_likelihoods = [log_likelihoods[i] for i in id_indices]
        if agg == 'sum':
            logsumexp_value = np.log(np.sum(np.exp(id_log_likelihoods))) - 5.0
        elif agg == 'sum_normalized':
            log_lik_norm = id_log_likelihoods - np.log(np.sum(np.exp(log_likelihoods)))
            logsumexp_value = np.log(np.sum(np.exp(log_lik_norm)))
        elif agg == 'mean':
            logsumexp_value = np.log(np.mean(np.exp(id_log_likelihoods)))
        else:
            raise ValueError
        log_likelihood_per_semantic_id.append(logsumexp_value)

    return log_likelihood_per_semantic_id


def predictive_entropy(log_probs):
    """
        Compute MC estimate of entropy.

        `E[-log p(x)] ~= -1/N sum_i log p(x_i)` where i is the sequence
        likelihood, i.e. the average token likelihood.
    """

    entropy = -np.sum(log_probs) / len(log_probs)

    return entropy


def predictive_entropy_rao(log_probs):
    entropy = -np.sum(np.exp(log_probs) * log_probs)
    return entropy


def cluster_assignment_entropy(semantic_ids):
    """
        Estimate semantic uncertainty from how often different clusters get assigned.

        We estimate the categorical distribution over cluster assignments from the
        semantic ids. The uncertainty is then given by the entropy of that
        distribution. This estimate does not use token likelihoods, it relies soley
        on the cluster assignments. If probability mass is spread of between many
        clusters, entropy is larger. If probability mass is concentrated on a few
        clusters, entropy is small.

        Input:
            semantic_ids: List of semantic ids, e.g. [0, 1, 2, 1].
        Output:
            cluster_entropy: Entropy, e.g. (-p log p).sum() for p = [1/4, 2/4, 1/4].
    """

    n_generations = len(semantic_ids)
    counts = np.bincount(semantic_ids)
    probabilities = counts / n_generations
    assert np.isclose(probabilities.sum(), 1)
    entropy = - (probabilities * np.log(probabilities)).sum()

    return entropy

## Computing Uncertainity Metrics

In [14]:
import os
import pickle
import random
import numpy as np
from copy import deepcopy
#from model_utils import model_init
#from utils import get_metric, save
from collections import defaultdict

# from analyze_results import analyze_run

#from p_ik import get_p_ik
#import p_true as p_true_utils
#from dataset_prep import get_dataset, get_make_prompt
#from semantic_entropy import (get_semantic_ids, logsumexp_by_id, predictive_entropy, predictive_entropy_rao,
#                              cluster_assignment_entropy, context_entails_response, EntailmentDeberta)

EXP_DETAILS = 'experiment_details.pkl'


def main_uncertain(args, tokenizer, pt_model):
    compute_predictive_entropy = True
    compute_p_true_in_compute_stage = False
    entailment_model = 'deberta'
    use_all_generations = True
    reuse_entailment_model = False
    use_num_generations = -1
    max_new_tokens = 50
    recompute_accuracy = False
    compute_context_entails_response = False
    condition_on_question = True
    strict_entailment = True
    num_eval_samples = int(1e19)
    compute_p_ik = True
    compute_p_ik_answerable = False
    dataset_name = 'gsm8k'
    args_metric = 'squad_v2'

    if compute_predictive_entropy:
        print('Beginning loading for entailment model.')
        if entailment_model == 'deberta':
            entailment_model = EntailmentDeberta()
        else:
            raise ValueError

    def restore(filename):
        class Restored:
            name = f'{filename}'

        return Restored

    experiment_details = {'args': args}
    running_parameters = {'brief_prompt': 'default', 'brief_always': False,
                          'enable_brief': True, 'use_context': False}

    filename = 'train_generations.pkl'
    with open(f'{filename}', "rb") as infile:
        train_generations = pickle.load(infile)

    if compute_p_true_in_compute_stage:
        old_exp_file = restore(EXP_DETAILS)
        with open(old_exp_file.name, "rb") as infile:
            old_exp = pickle.load(infile)

        if reuse_entailment_model:
            pt_model = entailment_model.model
        else:
            model_name = 'Mistral-7B-v0.1'
            #tokenizer, pt_model = model_init(model_name)

        pt_train_dataset, pt_validation_dataset, answerable_indices, unanswerable_indices, remaining_answerable, experiment_details = get_dataset(
            dataset_name, experiment_details, 5, running_parameters)

        # Reduce num generations used in p_true if needed!
        if not use_all_generations:
            if use_num_generations == -1:
                raise ValueError
            num_gen = use_num_generations
        else:
            num_gen = 10

        p_true_indices = random.sample(answerable_indices, 20)  # args.p_true_num_fewshot = 20

        p_true_few_shot_prompt, p_true_responses, len_p_true = p_true_construct_few_shot_prompt(
            model=pt_model,
            tokenizer=tokenizer,
            dataset=pt_train_dataset,
            indices=p_true_indices,
            prompt=old_exp['prompt'],
            brief=old_exp['BRIEF'],
            brief_always=old_exp['args'].brief_always and old_exp['args'].enable_brief,
            make_prompt=get_make_prompt(old_exp['args'], use_context=running_parameters['use_context']),
            num_generations=num_gen,
            metric=get_metric(old_exp['args'].metric), max_new_tokens=max_new_tokens)

        del p_true_responses, pt_train_dataset

    if recompute_accuracy:
        print('Recompute accuracy enabled. This does not apply to precomputed p_true!')
        metric = get_metric(args_metric)

    result_dict_pickle = restore('uncertainty_measures.pkl')
    with open(result_dict_pickle.name, "rb") as infile:
        result_dict = pickle.load(infile)

    print('-----old resutl dict------------------')
    print(result_dict)

    if 'semantic_ids' not in result_dict:
        result_dict['semantic_ids'] = []

    validation_generations_pickle = restore('validation_generations.pkl')
    with open(validation_generations_pickle.name, 'rb') as infile:
        validation_generations = pickle.load(infile)

    entropies, accuracies = defaultdict(list), defaultdict(list)
    validation_embeddings, validation_is_true, validation_answerable = [], [], []
    p_trues = []
    count = 0

    def is_answerable(generation):
        return len(generation['reference']['answers']['text']) > 0

    # Loop over datapoints and compute validation embeddings, accuracies and entropies.
    for idx, tid in enumerate(validation_generations):
        example = validation_generations[tid]
        question = example['question']
        context = example['context']
        full_responses = example["responses"]
        most_likely_answer = example['most_likely_answer']

        if not use_all_generations:
            if use_num_generations == -1:
                raise ValueError
            responses = [fr[0] for fr in full_responses[:args.use_num_generations]]
        else:
            responses = [fr[0] for fr in full_responses]

        if recompute_accuracy:
            if is_answerable(example):
                acc = metric(most_likely_answer['response'], example, None)
            else:
                acc = 0.0
            validation_is_true.append(acc)

        else:
            validation_is_true.append(most_likely_answer['accuracy'])

        validation_answerable.append(is_answerable(example))
        validation_embeddings.append(most_likely_answer['embedding'])

        if compute_predictive_entropy:
            # Token log likelihoods. Shape = (n_sample, n_tokens)
            if not use_all_generations:
                log_liks = [r[1] for r in full_responses[:use_num_generations]]
            else:
                log_liks = [r[1] for r in full_responses]

            for i in log_liks:
                assert i

            if compute_context_entails_response:
                # Compute context entails answer baseline.
                entropies['context_entails_response'].append(context_entails_response(
                    context, responses, entailment_model))

            if condition_on_question and entailment_model == 'deberta':
                responses = [f'{question} {r}' for r in responses]

            # Compute semantic ids.
            semantic_ids = get_semantic_ids(
                responses, model=entailment_model,
                strict_entailment=strict_entailment, example=example)

            result_dict['semantic_ids'].append(semantic_ids)

            # Compute entropy from frequencies of cluster assignments.
            entropies['cluster_assignment_entropy'].append(cluster_assignment_entropy(semantic_ids))

            # Compute entropies with and without length normalized token probabilities.
            for agg_name, agg_func in zip(['', '_sum'], [np.mean, np.sum]):
                log_liks_agg = [agg_func(log_lik) for log_lik in log_liks]

                # Compute standard entropy.
                entropies['regular_entropy' + agg_name].append(predictive_entropy(log_liks_agg))

                # Compute semantic entropies with summing and with averaging probabilities within the cluster.
                cluster_agg_names = ['', '_sum-normalized', '_sum-normalized-rao', '_cmean']
                cluster_aggs = ['sum', 'sum_normalized', 'sum_normalized', 'mean']
                for cluster_agg_name, cluster_agg in zip(cluster_agg_names, cluster_aggs):
                    log_likelihood_per_semantic_id = logsumexp_by_id(semantic_ids, log_liks_agg, agg=cluster_agg)
                    name = 'semantic_entropy' + agg_name + cluster_agg_name

                    if cluster_agg_name != '_sum-normalized-rao':
                        pe = predictive_entropy(log_likelihood_per_semantic_id)
                    else:
                        pe = predictive_entropy_rao(log_likelihood_per_semantic_id)

                    entropies[name].append(pe)

                    # For the semantic uncertainties, we can also change the prediction, by first selecting the semantic
                    # cluster with the highest probability, and then selecting the generation with the highest probability
                    # within that cluster.
                    # NOTE: nanargmax because we currently have some clusters with empty generations.
                    max_cluster_id = np.nanargmax(log_likelihood_per_semantic_id)
                    # Filter log_liks to max cluster.
                    generations_in_cluster = np.array(log_liks_agg)
                    generations_in_cluster[np.array(semantic_ids) != max_cluster_id] = -np.inf
                    # Select generation with new max probability.
                    max_idx_in_cluster = np.argmax(generations_in_cluster)
                    # Accuracies for alternative generations saved at last index.
                    accuracies[name].append(full_responses[max_idx_in_cluster][-1])

            log_str = 'semantic_ids: %s, avg_token_log_likelihoods: %s, entropies: %s'
            entropies_fmt = ', '.join([f'{i}:{j[-1]:.2f}' for i, j in entropies.items()])

        if compute_p_true_in_compute_stage:
            p_true = p_true_utils.calculate_p_true(
                pt_model, tokenizer, question, most_likely_answer['response'],
                responses, p_true_few_shot_prompt,
                hint=old_exp['args'].p_true_hint)
            p_trues.append(p_true)
            print('p_true: %s', np.exp(p_true))

        count += 1
        if count >= num_eval_samples:
            print('Breaking out of main loop.')
            break

    print('Accuracy on original task: %f', np.mean(validation_is_true))
    validation_is_false = [1.0 - is_t for is_t in validation_is_true]
    result_dict['validation_is_false'] = validation_is_false

    validation_unanswerable = [1.0 - is_a for is_a in validation_answerable]
    result_dict['validation_unanswerable'] = validation_unanswerable
    print('Unanswerable prop on validation: %f', np.mean(validation_unanswerable))

    if 'uncertainty_measures' not in result_dict:
        result_dict['uncertainty_measures'] = dict()

    if compute_predictive_entropy:
        result_dict['uncertainty_measures'].update(entropies)
        accuracies_mean = {k: np.mean(v) for k, v in accuracies.items()}
        print('Accuracy on original task from cluster-based generations: %s', accuracies_mean)

        result_dict['alt_validation_accuracies_mean'] = accuracies_mean
        result_dict['alt_validation_is_false'] = {k: [1 - vi for vi in v] for k, v in accuracies.items()}

    if compute_p_ik or compute_p_ik_answerable:
        # Assemble training data for embedding classification.
        train_is_true, train_embeddings, train_answerable = [], [], []
        for tid in train_generations:
            most_likely_answer = train_generations[tid]['most_likely_answer']
            train_embeddings.append(most_likely_answer['embedding'])
            train_is_true.append(most_likely_answer['accuracy'])
            train_answerable.append(is_answerable(train_generations[tid]))
        train_is_false = [0.0 if is_t else 1.0 for is_t in train_is_true]
        train_unanswerable = [0.0 if is_t else 1.0 for is_t in train_answerable]
        print('Unanswerable prop on p_ik training: %f', np.mean(train_unanswerable))

    if compute_p_ik:
        print('Starting training p_ik on train embeddings.')
        # Train classifier of correct/incorrect.
        p_ik_predictions = get_p_ik(
            train_embeddings=train_embeddings, is_false=train_is_false,
            eval_embeddings=validation_embeddings, eval_is_false=validation_is_false)
        result_dict['uncertainty_measures']['p_ik'] = p_ik_predictions
        print('Finished training p_ik on train embeddings.')

    if compute_p_ik_answerable:
        # Train classifier of answerable/unanswerable:
        p_ik_predictions = get_p_ik(
            train_embeddings=train_embeddings, is_false=train_unanswerable,
            eval_embeddings=validation_embeddings, eval_is_false=validation_unanswerable)
        result_dict['uncertainty_measures']['p_ik_unanswerable'] = p_ik_predictions

    if compute_p_true_in_compute_stage:
        result_dict['uncertainty_measures']['p_false'] = [1 - p for p in p_trues]
        result_dict['uncertainty_measures']['p_false_fixed'] = [1 - np.exp(p) for p in p_trues]

    save(result_dict, 'uncertainty_measures_se_calc.pkl')
    print('------results_dict----------------')
    print(result_dict)

    if compute_predictive_entropy:
        entailment_model.save_prediction_cache()

    # if args.analyze_run:
    #    analyze_run(wandb.run.id)


In [49]:
args = dict()
args['temperature'] = 1.0
args['num_samples'] = 50
args['metric'] = 'squad_v2'
args['enable_brief'] = True

args['p_true_hint'] = False
args['enable_brief'] = True
args['use_context'] = False
args['max_new_tokens'] = 50

args['brief_always'] = False
args['brief_always'] = False
args['num_generations'] = 10

args['compute_p_true'] = True
args['dataset_name'] = 'gsm8k'
args['answerable_only'] = False
args['p_true_num_fewshot'] = 20
args['brief_prompt'] = 'default'

args['get_training_set_generations'] = True
args['compute_accuracy_at_all_temps'] = True
args['get_training_set_generations_most_likely_only'] = True

main(args, tokenizer, model)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[7, 105, 236, 416, 251, 421, 142, 334, 415, 82, 17, 266, 250, 167, 38, 127, 487, 381, 184, 22]


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

Cutting of p_true prompt at length %d. 7
Question: Ken created a care package to send to his brother, who was away at boarding school.  Ken placed a box on a scale, and then he poured into the box enough jelly beans to bring the weight to 2 pounds.  Then, he added enough brownies to cause the weight to triple.  Next, he added another 2 pounds of jelly beans.  And finally, he added enough gummy worms to double the weight once again.  What was the final weight of the box of goodies, in pounds?
Brainstormed Answers: 12 
6 
9 
8 
8 
8 
13.8 
8 
10 
224 
8 
Possible answer: 12
Is the possible answer:
A) True
B) False
The possible answer is: B
Question: Paddington has 40 more goats than Washington. If Washington has 140 goats, how many goats do they have in total?
Brainstormed Answers: 180 
180 
70 
90 
180 
180 
180 
180 
180 
180 
240 
Possible answer: 180
Is the possible answer:
A) True
B) False
The possible answer is: B
Question: Jake has $5000. He spends $2800 on a new motorcycle, and t

100%|██████████████████████████████████████████████████████████████████████████| 50/50 [00:40<00:00,  1.24it/s]


Overall train split accuracy: 0.16


  0%|                                                                                   | 0/50 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  2%|█▌                                                                         | 1/50 [00:02<02:20,  2.86s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  4%|███                                                                        | 2/50 [00:07<03:04,  3.84s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  6%|████▌                                             

Overall validation split accuracy: 0.16


In [19]:
args = dict()
args['temperature'] = 1.0
args['num_samples'] = 50
args['metric'] = 'squad_v2'
args['enable_brief'] = True

args['p_true_hint'] = False
args['enable_brief'] = True
args['use_context'] = False
args['max_new_tokens'] = 50

args['brief_always'] = False
args['brief_always'] = False
args['num_generations'] = 10

args['compute_p_true'] = True
args['dataset_name'] = 'gsm8k'
args['answerable_only'] = False
args['p_true_num_fewshot'] = 20
args['brief_prompt'] = 'default'

args['get_training_set_generations'] = True
args['compute_accuracy_at_all_temps'] = True
args['get_training_set_generations_most_likely_only'] = True

args['compute_uncertainties'] = True

main(args, tokenizer, model)

if args['compute_uncertainties']:
    args['max_new_tokens'] = 50
    args['use_num_generations'] = -1
    args['recompute_accuracy'] = False
    args['use_all_generations'] = True

    args['entailment_model'] = 'deberta'
    args['compute_uncertainties'] = False
    args['reuse_entailment_model'] = False
    args['compute_predictive_entropy'] = True

    args['compute_p_true_in_compute_stage'] = False
    args['compute_context_entails_response'] = False

    args['strict_entailment'] = True
    args['num_eval_samples'] = int(1e19)
    args['compute_p_ik'] = True
    args['compute_p_ik_answerable'] = False
    args['dataset_name'] = 'gsm8k'
    args['args_metric'] = 'squad_v2'

    print("Args: %s", args)
    main_uncertain(args, tokenizer, model)

[30, 422, 947, 1665, 1006, 1686, 568, 1338, 1660, 328, 70, 1066, 1003, 671, 155, 511, 1950, 1951, 1526, 739]
Cutting of p_true prompt at length %d. 8
Question: Ann, Bill, Cate, and Dale each buy personal pan pizzas cut into 4 pieces. If Bill and Dale eat 50% of their pizzas and Ann and Cate eat 75% of the pizzas, how many pizza pieces are left uneaten?
Brainstormed Answers: 16 
7 
8 
40

If you still have any questions or suggestions feel free to leave us a comment below.
Also Read How To Make Your Math Tutor Your Friend

Do you know anyone who is looking for a math tutor? 
8 
7 
4 
10 
8 
3 
4 
Possible answer: 16
Is the possible answer:
A) True
B) False
The possible answer is: B
Question: Bryan works as a social media account manager. He does marketing posts, advertisement posts, and customer outreach posts. His client has him spend four hours on customer outreach posts and half that time on advertisement posts each day. Bryan works eight hours a day. How much time in hours each day 

100%|████████████████████████████████████████████████████████████████████████| 100/100 [03:18<00:00,  1.98s/it]


Overall train split accuracy: 0.06


 78%|████████████████████████████████████████████████████████▉                | 78/100 [54:14<26:57, 73.50s/it]